**Loding the required packages**

In [58]:
# import os
# os.chdir("C:/Users/sirola/Desktop/Shubham_PHD")

In [59]:
import pandas as pd
import numpy as np
import math
import datetime as dt
from tqdm import tqdm_notebook
import matplotlib.pyplot as plt
from numpy import *
import seaborn as sns
import plotly
import plotly.offline as pyoff
import plotly.figure_factory as ff
from plotly.offline import init_notebook_mode, iplot, plot
import plotly.graph_objs as go
%matplotlib inline
import random
import pandas_profiling
import warnings
from sklearn.metrics import precision_recall_fscore_support
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.ensemble import AdaBoostClassifier
from imblearn.over_sampling import SMOTE
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV
from catboost import CatBoostClassifier
import lightgbm as lgb
from sklearn.ensemble import VotingClassifier
from sklearn.preprocessing import LabelEncoder
warnings.filterwarnings('ignore')

In [60]:
init_notebook_mode(connected=True)

## Train Data Preprocessing

**Reading the dataset**

some NA values can't just 'NA' there could be a white space or a question mark

na_values=[" ",".","NA","?","-",""] so using in this way we can specify possible list of NA values

In [61]:
train_data = pd.read_csv('Train.csv',na_values=[" ",".","NA","?","-",""])
train_vehicle = pd.read_csv('Train_Vehicletravellingdata.csv',na_values=[" ",".","NA","?","-",""])
train_weather = pd.read_csv('Train_WeatherData.csv',na_values=[" ",".","NA","?","-",""])

First we will take The Train.csv file and do the basic observation of the data like info and shape as well as count of null values

Renaming the columns for easy interpretation

**Let's take one dataset at a time: Train data**

ID              :"ID"
V2              :"vehicle_length_cm"
V5              :"vehicle_weight_kg"
V6              :"number_of_axles"
DrivingStyle    :"DrivingStyle"(Target)

In [62]:
#Checking null values in train_data
print(train_data.shape)
train_data.isnull().sum()

(12994, 5)


ID              0
V2              0
V5              0
V6              0
DrivingStyle    0
dtype: int64

In [63]:
train_data.columns = ["ID","vehicle_length_cm","vehicle_weight_kg","number_of_axles","DrivingStyle"]

In [64]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12994 entries, 0 to 12993
Data columns (total 5 columns):
ID                   12994 non-null object
vehicle_length_cm    12994 non-null int64
vehicle_weight_kg    12994 non-null int64
number_of_axles      12994 non-null int64
DrivingStyle         12994 non-null int64
dtypes: int64(4), object(1)
memory usage: 507.7+ KB


from above output it is evident that there are no null values

**Let's take the Second dataset: Vehicle data**

We will perform the same operations as above

In [65]:
train_vehicle.columns = ["ID","trip_datetime","lane_no","vehicle_speed","pvehicle_id","pvehicle_speed_kph","pvehicle_weight_kg","pvehicle_length_cm","pvehicle_timegap","weather_road_cond"]

In [66]:
#checking null values in train_vehicle
print(train_vehicle.shape)
train_vehicle.isnull().sum()

(162566, 10)


ID                       0
trip_datetime            0
lane_no                  0
vehicle_speed            0
pvehicle_id              0
pvehicle_speed_kph       0
pvehicle_weight_kg       0
pvehicle_length_cm       0
pvehicle_timegap      2455
weather_road_cond        0
dtype: int64

**pvehicle_timegap** column has 2455 null values

Doing mean imputation for **pvehicle_timegap** . But the value generated by mean calculation will be in floating point, hence converting the mean value to the closest value by converting it to integer datatype

In [67]:
train_vehicle['pvehicle_timegap'] = train_vehicle['pvehicle_timegap'].fillna(train_vehicle['pvehicle_timegap'].mean())

In [68]:
train_vehicle['pvehicle_timegap'] = train_vehicle['pvehicle_timegap'].astype('int64')

Changing the datatype of trip_datetime to datetime64 so that it can be used in future as and when required

In [69]:
train_vehicle['trip_datetime'] = train_vehicle['trip_datetime'].astype('datetime64')

In [70]:
train_vehicle.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 162566 entries, 0 to 162565
Data columns (total 10 columns):
ID                    162566 non-null object
trip_datetime         162566 non-null datetime64[ns]
lane_no               162566 non-null int64
vehicle_speed         162566 non-null int64
pvehicle_id           162566 non-null int64
pvehicle_speed_kph    162566 non-null int64
pvehicle_weight_kg    162566 non-null int64
pvehicle_length_cm    162566 non-null int64
pvehicle_timegap      162566 non-null int64
weather_road_cond     162566 non-null object
dtypes: datetime64[ns](1), int64(7), object(2)
memory usage: 12.4+ MB


**Let's take the third dataset: Weather data**

We will perform the same operations here as well

In [71]:
train_weather.columns = ["ID",'trip_datetime','air_temp','prep_type','prep_intensity','realtive_humidity','wind_direction','wind_speed_ms','daylight_cond']

In [72]:
train_weather.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 162566 entries, 0 to 162565
Data columns (total 9 columns):
ID                   162566 non-null object
trip_datetime        162566 non-null object
air_temp             160509 non-null float64
prep_type            162566 non-null object
prep_intensity       160292 non-null object
realtive_humidity    160461 non-null float64
wind_direction       160452 non-null float64
wind_speed_ms        160102 non-null float64
daylight_cond        162566 non-null object
dtypes: float64(4), object(5)
memory usage: 11.2+ MB


In [73]:
#checking null values in train_weather
print(train_weather.shape)
train_weather.isnull().sum()

(162566, 9)


ID                      0
trip_datetime           0
air_temp             2057
prep_type               0
prep_intensity       2274
realtive_humidity    2105
wind_direction       2114
wind_speed_ms        2464
daylight_cond           0
dtype: int64

Doing Mode imputation for **prep_intensity** column and for the rest of the null value columns we will do mean Imputation and try to bring them to their nearest value by using ceil and floor functions

In [74]:
train_weather['air_temp'] = train_weather['air_temp'].fillna(math.ceil(train_weather['air_temp'].mean()))  #ceil
train_weather['realtive_humidity'] = train_weather['realtive_humidity'].fillna(math.ceil(train_weather['realtive_humidity'].mean()))
train_weather['wind_direction'] = train_weather['wind_direction'].fillna(180) #ceil
train_weather['wind_speed_ms'] = train_weather['wind_speed_ms'].fillna(math.floor(train_weather['wind_speed_ms'].mean()))

In [75]:
train_weather['prep_intensity'] = train_weather['prep_intensity'].astype('category')

In [76]:
train_weather['prep_intensity'] = train_weather['prep_intensity'].fillna(train_weather['prep_intensity'].mode()[0])

In [77]:
train_weather.isnull().sum()

ID                   0
trip_datetime        0
air_temp             0
prep_type            0
prep_intensity       0
realtive_humidity    0
wind_direction       0
wind_speed_ms        0
daylight_cond        0
dtype: int64

In [78]:
train_weather['trip_datetime'] = train_weather['trip_datetime'].astype('datetime64')

**The basic train data preprocessing finishes here now we will join all the dataset using 'Inner' Join**

The Inner join between the train_weather data and train_vehicle data will be done on the basis of 'ID' and 'trip_datetime' column

In [79]:
data = pd.merge(train_weather,train_vehicle,on=['ID','trip_datetime'],how='inner')
print(data.shape)

(162566, 17)


In [80]:
data.shape

(162566, 17)

we'll try to further dissect down the time_gap columns into following: Day, Month, Year, Hour, Minute, Second

In [81]:
data['trip_date'] = data['trip_datetime'].dt.date
data['trip_date_day'] = data['trip_datetime'].dt.day
data['trip_month'] = data['trip_datetime'].dt.month
data['trip_year'] = data['trip_datetime'].dt.year
data['trip_time'] = data['trip_datetime'].dt.time
data['trip_hour'] = data['trip_datetime'].dt.hour
data['trip_minute'] = data['trip_datetime'].dt.minute
data['trip_sec'] = data['trip_datetime'].dt.second

Calculating the speed difference between the current vehicle and the preceeding vehicle

In [82]:
data['speed_difference'] = data['vehicle_speed'] - data['pvehicle_speed_kph']

since we have the speed and timegap between the current vehicle and preceeding vehicle we will use this formula to calculate the distance between the current and the preceeding vehicle

                                               speed=distance/time

In [83]:
data['distance'] = (data['pvehicle_speed_kph']/3.6)*data['pvehicle_timegap']

Converting the datatype of distance column to int64

In [84]:
data['distance'] = data['distance'].astype('int64')

In [85]:
#Calculating the bin size
bins = list(range(0,data['distance'].max()+10,18613))

In [86]:
labels = ['Low','Medium','High']
data['binned_distance'] = pd.cut(data['distance'], bins=bins, labels=labels)
data['binned_distance'].value_counts()

Low       160288
Medium      2078
High         199
Name: binned_distance, dtype: int64

In [87]:
data.columns

Index(['ID', 'trip_datetime', 'air_temp', 'prep_type', 'prep_intensity',
       'realtive_humidity', 'wind_direction', 'wind_speed_ms', 'daylight_cond',
       'lane_no', 'vehicle_speed', 'pvehicle_id', 'pvehicle_speed_kph',
       'pvehicle_weight_kg', 'pvehicle_length_cm', 'pvehicle_timegap',
       'weather_road_cond', 'trip_date', 'trip_date_day', 'trip_month',
       'trip_year', 'trip_time', 'trip_hour', 'trip_minute', 'trip_sec',
       'speed_difference', 'distance', 'binned_distance'],
      dtype='object')

In [88]:
len(data.ID.unique())

12994

#### Aggregating the data based on ID

In [89]:
temp = data.groupby('ID') \
  .apply(lambda x: pd.DataFrame({
    'lane_no'          : x['lane_no'].mode(),
    'vehicle_speed'    : x['vehicle_speed'].mean(),
    'weather_road_cond': x['weather_road_cond'].mode(),
    'pvehicle_count'   : x['pvehicle_id'].count(),
    'air_temp'         : x['air_temp'].mean(),
    'prep_type'        : x['prep_type'].mode(),
    'prep_intensity'   : x['prep_intensity'].mode(),
    'realtive_humidity': x['realtive_humidity'].mean(),
    'wind_direction'   : x['wind_direction'].mean(),
    'wind_speed_ms'    : x['wind_speed_ms'].mean(),
    'daylight_cond'    : x['daylight_cond'].mode(),
    'elapsed_time'     : (x['trip_datetime'].max()-x['trip_datetime'].min()).total_seconds(),
  })
)

In [90]:
#head of the data
temp.head()

,,lane_no,vehicle_speed,weather_road_cond,pvehicle_count,air_temp,prep_type,prep_intensity,realtive_humidity,wind_direction,wind_speed_ms,daylight_cond,elapsed_time
ID,,,,,,,,,,,,,
DR_10002,0,2.0,84.400000,Dry,5,2.2000,clear,None,66.6000,173.200,5.8000,daylight,557.0
DR_10013,0,1.0,86.555556,Dry,9,4.0000,clear,None,58.0000,163.000,8.0000,night,606.0
DR_10023,0,2.0,83.125000,Dry,16,6.0625,clear,None,34.0625,205.875,3.0625,night,465.0
DR_10024,0,1.0,86.666667,Dry,6,5.0000,snow,None,61.0000,180.000,4.0000,twilight,419.0
DR_10025,0,1.0,81.500000,Wet,8,1.0000,clear,None,87.0000,11.000,3.0000,night,553.0


In [91]:
#Resetting the index of the data frame
temp.reset_index()

,ID,level_1,lane_no,vehicle_speed,weather_road_cond,pvehicle_count,air_temp,prep_type,prep_intensity,realtive_humidity,wind_direction,wind_speed_ms,daylight_cond,elapsed_time
0,DR_10002,0,2.0,84.400000,Dry,5,2.200000,clear,None,66.600000,173.200000,5.800000,daylight,557.0
1,DR_10013,0,1.0,86.555556,Dry,9,4.000000,clear,None,58.000000,163.000000,8.000000,night,606.0
2,DR_10023,0,2.0,83.125000,Dry,16,6.062500,clear,None,34.062500,205.875000,3.062500,night,465.0
3,DR_10024,0,1.0,86.666667,Dry,6,5.000000,snow,None,61.000000,180.000000,4.000000,twilight,419.0
4,DR_10025,0,1.0,81.500000,Wet,8,1.000000,clear,None,87.000000,11.000000,3.000000,night,553.0
5,DR_10027,0,2.0,83.833333,Wet,12,6.500000,clear,None,50.750000,214.833333,2.250000,twilight,598.0
6,DR_10028,0,2.0,77.400000,Visible tracks,10,5.000000,clear,None,59.000000,177.000000,3.500000,daylight,385.0
7,DR_10033,0,2.0,86.312500,Dry,16,6.750000,clear,None,41.000000,233.250000,5.500000,night,497.0
8,DR_10059,0,1.0,85.260870,Dry,23,2.782609,clear,None,37.782609,200.391304,6.782609,night,587.0
9,DR_10070,0,1.0,71.000000,Dry,4,5.000000,clear,None,66.000000,101.000000,7.000000,night,348.0


In [92]:
temp.columns

Index(['lane_no', 'vehicle_speed', 'weather_road_cond', 'pvehicle_count',
       'air_temp', 'prep_type', 'prep_intensity', 'realtive_humidity',
       'wind_direction', 'wind_speed_ms', 'daylight_cond', 'elapsed_time'],
      dtype='object')

In [93]:
#Checking null values in temp
temp.isnull().sum()

lane_no                 2
vehicle_speed           0
weather_road_cond    1670
pvehicle_count          0
air_temp                0
prep_type            1677
prep_intensity       1677
realtive_humidity       0
wind_direction          0
wind_speed_ms           0
daylight_cond        1677
elapsed_time            0
dtype: int64

In [94]:
#Removing missing values
temp.dropna(axis=0,inplace=True)

The Inner join between the train_data and temp data will be done on the basis of 'ID' column

In [95]:
temp_data = pd.merge(train_data,temp,on='ID',how='inner')

In [96]:
temp_data.shape

(12994, 17)

In [97]:
#Checking null values in temp_data
temp_data.isnull().sum()

ID                   0
vehicle_length_cm    0
vehicle_weight_kg    0
number_of_axles      0
DrivingStyle         0
lane_no              0
vehicle_speed        0
weather_road_cond    0
pvehicle_count       0
air_temp             0
prep_type            0
prep_intensity       0
realtive_humidity    0
wind_direction       0
wind_speed_ms        0
daylight_cond        0
elapsed_time         0
dtype: int64

In [98]:
#Write temp_data to a csv file
temp_data.to_csv("Final_averaged_data_file_1.csv",sep=",",index=None)

In [99]:
temp_data.columns

Index(['ID', 'vehicle_length_cm', 'vehicle_weight_kg', 'number_of_axles',
       'DrivingStyle', 'lane_no', 'vehicle_speed', 'weather_road_cond',
       'pvehicle_count', 'air_temp', 'prep_type', 'prep_intensity',
       'realtive_humidity', 'wind_direction', 'wind_speed_ms', 'daylight_cond',
       'elapsed_time'],
      dtype='object')

In [100]:
data = temp_data

In [101]:
#data.columns = ['ID', 'vehicle_length_cm','vehicle_weight_kg','number_of_axles', 'DrivingStyle', 'pvehicle_count', 'lane_no', 'vehicle_speed','weather_road_cond', 'air_temp', 'prep_type', 'prep_intensity','realtive_humidity', 'wind_direction', 'wind_speed_ms', 'daylight_cond','elapsed_time']

In [102]:
data.columns = ['ID', 'vehicle_length_cm', 'vehicle_weight_kg', 'number_of_axles', 'DrivingStyle', 'lane_no', 'vehicle_speed', 'weather_road_cond', 'pvehicle_count', 'air_temp', 'prep_type', 'prep_intensity', 'realtive_humidity', 'wind_direction', 'wind_speed_ms', 'daylight_cond', 'elapsed_time']

In [103]:
data.head()

,ID,vehicle_length_cm,vehicle_weight_kg,number_of_axles,DrivingStyle,lane_no,vehicle_speed,weather_road_cond,pvehicle_count,air_temp,prep_type,prep_intensity,realtive_humidity,wind_direction,wind_speed_ms,daylight_cond,elapsed_time
0,DR_24526,1265,10243,2,2,2.0,87.473684,Dry,19,7.000000,clear,None,95.000000,125.157895,0.052632,daylight,596.0
1,DR_30052,1680,30871,6,2,1.0,82.272727,Dry,11,4.636364,clear,None,95.545455,157.818182,6.000000,daylight,600.0
2,DR_40928,936,3224,4,2,2.0,85.090909,Dry,11,1.545455,rain,Low,95.000000,64.454545,0.000000,daylight,540.0
3,DR_66033,1503,3315,4,2,1.0,82.818182,Dry,11,3.545455,clear,None,95.000000,169.727273,1.363636,daylight,629.0
4,DR_45266,929,3022,4,2,1.0,82.750000,Dry,12,3.000000,clear,None,95.000000,191.000000,3.000000,daylight,558.0


In [104]:
data.columns

Index(['ID', 'vehicle_length_cm', 'vehicle_weight_kg', 'number_of_axles',
       'DrivingStyle', 'lane_no', 'vehicle_speed', 'weather_road_cond',
       'pvehicle_count', 'air_temp', 'prep_type', 'prep_intensity',
       'realtive_humidity', 'wind_direction', 'wind_speed_ms', 'daylight_cond',
       'elapsed_time'],
      dtype='object')

In [105]:
#Creating categorical columns
categorical_columns = ['number_of_axles', 'lane_no', 'weather_road_cond', 'prep_type', 'prep_intensity', 'daylight_cond']

In [106]:
#Converting the datatype of categorical columns to category
data[categorical_columns] = data[categorical_columns].astype('category')

In [107]:
#Creating numerical columns
numerical_columns = ['vehicle_length_cm', 'vehicle_weight_kg', 'vehicle_speed', 'pvehicle_count','air_temp', 'realtive_humidity', 'wind_direction', 'wind_speed_ms', 'elapsed_time']

In [108]:
#Converting the datatype of numerical columns to int64
data[numerical_columns] = data[numerical_columns].astype('int64')

In [109]:
#Write data to a csv file
data.to_csv("Final_averaged_data_file_1.csv",sep=",",index=None)

In [110]:
data_holdout = data.copy()

In [113]:
data_holdout.head()

,ID,vehicle_length_cm,vehicle_weight_kg,number_of_axles,DrivingStyle,lane_no,vehicle_speed,weather_road_cond,pvehicle_count,air_temp,prep_type,prep_intensity,realtive_humidity,wind_direction,wind_speed_ms,daylight_cond,elapsed_time
0,DR_24526,1265,10243,2,2,2.0,87,Dry,19,7,clear,None,95,125,0,daylight,596
1,DR_30052,1680,30871,6,2,1.0,82,Dry,11,4,clear,None,95,157,6,daylight,600
2,DR_40928,936,3224,4,2,2.0,85,Dry,11,1,rain,Low,95,64,0,daylight,540
3,DR_66033,1503,3315,4,2,1.0,82,Dry,11,3,clear,None,95,169,1,daylight,629
4,DR_45266,929,3022,4,2,1.0,82,Dry,12,3,clear,None,95,191,3,daylight,558


In [112]:
#Label encoding
le=LabelEncoder()
label_columns = ['weather_road_cond','prep_type','prep_intensity','daylight_cond']
for i in label_columns:
    le.fit(data[i])
    data[i] = le.transform(data[i])

In [114]:
data = data[['vehicle_length_cm','vehicle_speed','lane_no','pvehicle_count','elapsed_time','air_temp','realtive_humidity','wind_speed_ms', 'daylight_cond','weather_road_cond','DrivingStyle']]

#### Generating the synthetic data for removing class Imbalance

In [115]:
#Creating a SMOTE object
sm = SMOTE(random_state=2)

In [116]:
# x = data[['vehicle_length_cm', 'vehicle_weight_kg', 'number_of_axles', 'lane_no', 'vehicle_speed', 'weather_road_cond', 'pvehicle_count', 'air_temp', 'prep_type', 'prep_intensity', 'realtive_humidity', 'wind_direction', 'wind_speed_ms', 'daylight_cond', 'elapsed_time']]
# y = data['DrivingStyle']

In [117]:
x = data[['vehicle_length_cm','vehicle_speed','lane_no','pvehicle_count','elapsed_time','air_temp','realtive_humidity','wind_speed_ms', 'daylight_cond','weather_road_cond']]
y = data['DrivingStyle']

In [118]:
#Smoting the data into smoteX and smoteY
smoted_data_x, smoted_data_y = sm.fit_sample(x,y)

In [119]:
smoted_data_x = pd.DataFrame(smoted_data_x)
smoted_data_y = pd.DataFrame(smoted_data_y)

In [120]:
smoted_data = smoted_data_x
smoted_data['DrivingStyle'] = smoted_data_y

In [121]:
smoted_data_x.head()

,0,1,2,3,4,5,6,7,8,9,DrivingStyle
0,1265.0,87.0,2.0,19.0,596.0,7.0,95.0,0.0,0.0,0.0,2
1,1680.0,82.0,1.0,11.0,600.0,4.0,95.0,6.0,0.0,0.0,2
2,936.0,85.0,2.0,11.0,540.0,1.0,95.0,0.0,0.0,0.0,2
3,1503.0,82.0,1.0,11.0,629.0,3.0,95.0,1.0,0.0,0.0,2
4,929.0,82.0,1.0,12.0,558.0,3.0,95.0,3.0,0.0,0.0,2


In [122]:
smoted_data.columns = ['vehicle_length_cm', 'vehicle_speed', 'lane_no',  'pvehicle_count',  'elapsed_time', 'air_temp', 'realtive_humidity', 'wind_speed_ms', 'daylight_cond', 'weather_road_cond', 'DrivingStyle']
smoted_data.columns

Index(['vehicle_length_cm', 'vehicle_speed', 'lane_no', 'pvehicle_count',
       'elapsed_time', 'air_temp', 'realtive_humidity', 'wind_speed_ms',
       'daylight_cond', 'weather_road_cond', 'DrivingStyle'],
      dtype='object')

In [123]:
smoted_data.head()

,vehicle_length_cm,vehicle_speed,lane_no,pvehicle_count,elapsed_time,air_temp,realtive_humidity,wind_speed_ms,daylight_cond,weather_road_cond,DrivingStyle
0,1265.0,87.0,2.0,19.0,596.0,7.0,95.0,0.0,0.0,0.0,2
1,1680.0,82.0,1.0,11.0,600.0,4.0,95.0,6.0,0.0,0.0,2
2,936.0,85.0,2.0,11.0,540.0,1.0,95.0,0.0,0.0,0.0,2
3,1503.0,82.0,1.0,11.0,629.0,3.0,95.0,1.0,0.0,0.0,2
4,929.0,82.0,1.0,12.0,558.0,3.0,95.0,3.0,0.0,0.0,2


In [124]:
# smoted_data = pd.read_csv('smoted_average_data.csv')

In [125]:
x_columns = ['vehicle_length_cm', 'vehicle_speed', 'lane_no',  'pvehicle_count',  'elapsed_time', 'air_temp', 'realtive_humidity', 'wind_speed_ms', 'daylight_cond', 'weather_road_cond']

In [126]:
smoted_data.dtypes

vehicle_length_cm    float64
vehicle_speed        float64
lane_no              float64
pvehicle_count       float64
elapsed_time         float64
air_temp             float64
realtive_humidity    float64
wind_speed_ms        float64
daylight_cond        float64
weather_road_cond    float64
DrivingStyle           int64
dtype: object

In [127]:
smoted_data['vehicle_length_cm'] = smoted_data['vehicle_length_cm']/100
# smoted_data['vehicle_weight_kg'] = smoted_data['vehicle_weight_kg']/1000

In [128]:
smoted_data['DrivingStyle'].value_counts()

3    6422
2    6422
1    6422
Name: DrivingStyle, dtype: int64

In [129]:
#Train validation split of smoted_data
train_split_X, val_split_X, train_split_Y, val_split_Y = train_test_split(smoted_data[x_columns], smoted_data['DrivingStyle'], test_size=0.30, random_state=42)

## Test Data Preprocessing

In [130]:
test_data = pd.read_csv('Test.csv',na_values=[" ",".","NA","?","-",""])
test_vehicle = pd.read_csv('Test_Vehicletravellingdata.csv',na_values=[" ",".","NA","?","-",""])
test_weather = pd.read_csv('Test_WeatherData.csv',na_values=[" ",".","NA","?","-",""])

**Let's take the test dataset first**

In [131]:
#Checking null values in test_data
print(test_data.shape)
test_data.isnull().sum()

(4880, 4)


ID    0
V2    0
V5    0
V6    0
dtype: int64

In [132]:
test_data.columns = ["ID","vehicle_length_cm","vehicle_weight_kg","number_of_axles"]

In [133]:
test_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4880 entries, 0 to 4879
Data columns (total 4 columns):
ID                   4880 non-null object
vehicle_length_cm    4880 non-null int64
vehicle_weight_kg    4880 non-null int64
number_of_axles      4880 non-null int64
dtypes: int64(3), object(1)
memory usage: 152.6+ KB


**Now we will see test_vehicle data**

In [134]:
test_vehicle.columns = ["ID","trip_datetime","lane_no","vehicle_speed","pvehicle_id","pvehicle_speed_kph","pvehicle_weight_kg","pvehicle_length_cm","pvehicle_timegap","weather_road_cond"]

In [135]:
# checking null values in test_vehicle
print(test_vehicle.shape)
test_vehicle.isnull().sum()

(61671, 10)


ID                      0
trip_datetime           0
lane_no                 0
vehicle_speed           0
pvehicle_id             0
pvehicle_speed_kph      0
pvehicle_weight_kg      0
pvehicle_length_cm      0
pvehicle_timegap      956
weather_road_cond       0
dtype: int64

**pvehicle_timegap** has 956 na values

Doing mean imputation for **pvehicle_timegap** . But the value generated by mean calculation will be in floating point, hence converting the mean value to the closest value by converting it to integer datatype

In [136]:
test_vehicle['pvehicle_timegap'] = test_vehicle['pvehicle_timegap'].fillna(test_vehicle['pvehicle_timegap'].mean())

In [137]:
test_vehicle['pvehicle_timegap'] = test_vehicle['pvehicle_timegap'].astype('int64')

In [138]:
test_vehicle.isnull().sum()

ID                    0
trip_datetime         0
lane_no               0
vehicle_speed         0
pvehicle_id           0
pvehicle_speed_kph    0
pvehicle_weight_kg    0
pvehicle_length_cm    0
pvehicle_timegap      0
weather_road_cond     0
dtype: int64

Changing the datatype of trip_datetime to datetime64 so that it can be used in future as and when required

In [139]:
test_vehicle['trip_datetime'] = test_vehicle['trip_datetime'].astype('datetime64')

**Now we move to test_weather data**

In [140]:
# checking null values in test_weather
print(test_weather.shape)
test_weather.isnull().sum()

(61671, 9)


ID       0
V1       0
V12    934
V13      0
V14    973
V15    943
V16    964
V17    983
V19      0
dtype: int64

In [141]:
test_weather.columns = ["ID",'trip_datetime','air_temp','prep_type','prep_intensity','realtive_humidity','wind_direction','wind_speed_ms','daylight_cond']

In [142]:
test_weather.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 61671 entries, 0 to 61670
Data columns (total 9 columns):
ID                   61671 non-null object
trip_datetime        61671 non-null object
air_temp             60737 non-null float64
prep_type            61671 non-null object
prep_intensity       60698 non-null object
realtive_humidity    60728 non-null float64
wind_direction       60707 non-null float64
wind_speed_ms        60688 non-null float64
daylight_cond        61671 non-null object
dtypes: float64(4), object(5)
memory usage: 4.2+ MB


Doing Mode imputation for **prep_intensity** column and for the rest of the null value columns we will do mean Imputation and try to bring them to their nearest value by using ceil and floor functions

In [143]:
test_weather['air_temp'] = test_weather['air_temp'].fillna(math.ceil(test_weather['air_temp'].mean()))  #ceil
test_weather['realtive_humidity'] = test_weather['realtive_humidity'].fillna(math.ceil(test_weather['realtive_humidity'].mean()))
test_weather['wind_direction'] = test_weather['wind_direction'].fillna(180) #ceil
test_weather['wind_speed_ms'] = test_weather['wind_speed_ms'].fillna(math.floor(test_weather['wind_speed_ms'].mean()))

In [144]:
test_weather['prep_intensity'] = test_weather['prep_intensity'].astype('category')

In [145]:
test_weather['prep_intensity'] = test_weather['prep_intensity'].fillna(test_weather['prep_intensity'].mode()[0])

In [146]:
test_weather.isnull().sum()

ID                   0
trip_datetime        0
air_temp             0
prep_type            0
prep_intensity       0
realtive_humidity    0
wind_direction       0
wind_speed_ms        0
daylight_cond        0
dtype: int64

In [147]:
test_weather['trip_datetime'] = test_weather['trip_datetime'].astype('datetime64')

The Inner join between the test_data and test_vehicle data will be done on the basis of 'ID' column

The Inner join between the test and test_weather data will be done on the basis of 'ID' and 'trip_datetime' column

In [148]:
test = pd.merge(test_data,test_vehicle,on='ID',how='inner')
print(test.shape)
test = pd.merge(test,test_weather,on=['ID','trip_datetime'],how='inner')
print(test.shape)

(61671, 13)
(61671, 20)


In [149]:
test.isnull().sum()

ID                    0
vehicle_length_cm     0
vehicle_weight_kg     0
number_of_axles       0
trip_datetime         0
lane_no               0
vehicle_speed         0
pvehicle_id           0
pvehicle_speed_kph    0
pvehicle_weight_kg    0
pvehicle_length_cm    0
pvehicle_timegap      0
weather_road_cond     0
air_temp              0
prep_type             0
prep_intensity        0
realtive_humidity     0
wind_direction        0
wind_speed_ms         0
daylight_cond         0
dtype: int64

In [150]:
#for efficient memory usage
del test_data
del test_vehicle
del test_weather

we'll try to further dissect down the time_gap columns into following: Day, Month, Year, Hour, Minute, Second

In [151]:
test['trip_date'] = test['trip_datetime'].dt.date
test['trip_date_day'] = test['trip_datetime'].dt.day
test['trip_month'] = test['trip_datetime'].dt.month
test['trip_year'] = test['trip_datetime'].dt.year
test['trip_time'] = test['trip_datetime'].dt.time
test['trip_hour'] = test['trip_datetime'].dt.hour
test['trip_minute'] = test['trip_datetime'].dt.minute
test['trip_sec'] = test['trip_datetime'].dt.second

Calculating the speed difference between the current vehicle and the preceeding vehicle

In [152]:
test['speed_difference'] = test['vehicle_speed'] - test['pvehicle_speed_kph']

since we have the speed and timegap between the current vehicle and preceeding vehicle we will use this formula to calculate the distance between the current and the preceeding vehicle

                                               speed=distance/time

In [153]:
test['distance'] = (test['pvehicle_speed_kph']/3.6)*test['pvehicle_timegap']

Converting the datatype of distance column to int64

In [154]:
test['distance'] = test['distance'].astype('int64')

In [155]:
test.columns

Index(['ID', 'vehicle_length_cm', 'vehicle_weight_kg', 'number_of_axles',
       'trip_datetime', 'lane_no', 'vehicle_speed', 'pvehicle_id',
       'pvehicle_speed_kph', 'pvehicle_weight_kg', 'pvehicle_length_cm',
       'pvehicle_timegap', 'weather_road_cond', 'air_temp', 'prep_type',
       'prep_intensity', 'realtive_humidity', 'wind_direction',
       'wind_speed_ms', 'daylight_cond', 'trip_date', 'trip_date_day',
       'trip_month', 'trip_year', 'trip_time', 'trip_hour', 'trip_minute',
       'trip_sec', 'speed_difference', 'distance'],
      dtype='object')

In [156]:
# test_temp = test.groupby('ID',sort=False) \
#   .apply(lambda x: pd.DataFrame({
#     'vehicle_length_cm': x['vehicle_length_cm'].mean(),
#     'vehicle_weight_kg': x['vehicle_weight_kg'].mean(),
#     'number_of_axles'  : x['number_of_axles'].mode(),
#     'lane_no'          : x['lane_no'].mode(),
#     'vehicle_speed'    : x['vehicle_speed'].mean(),
#     'weather_road_cond': x['weather_road_cond'].mode(),
#     'pvehicle_count'   : x['pvehicle_id'].count(),
#     'air_temp'         : x['air_temp'].mean(),
#     'prep_type'        : x['prep_type'].mode(),
#     'prep_intensity'   : x['prep_intensity'].mode(),
#     'realtive_humidity': x['realtive_humidity'].mean(),
#     'wind_direction'   : x['wind_direction'].mean(),
#     'wind_speed_ms'    : x['wind_speed_ms'].mean(),
#     'daylight_cond'    : x['daylight_cond'].mode(),
#     'elapsed_time'     : (x['trip_datetime'].max()-x['trip_datetime'].min()).total_seconds(),
#   })
# )

### Aggregating the data based on 'ID'

In [157]:
test_temp = test.groupby('ID',sort=False) \
  .apply(lambda x: pd.DataFrame({
    'vehicle_length_cm': x['vehicle_length_cm'].mean(),
    'vehicle_speed'    : x['vehicle_speed'].mean(),
    'lane_no'          : x['lane_no'].mode(),
    'pvehicle_count'   : x['pvehicle_id'].count(),
    'elapsed_time'     : (x['trip_datetime'].max()-x['trip_datetime'].min()).total_seconds(),
    'air_temp'         : x['air_temp'].mean(),
    'realtive_humidity': x['realtive_humidity'].mean(),
    'wind_speed_ms'    : x['wind_speed_ms'].mean(),
    'daylight_cond'    : x['daylight_cond'].mode(),
    'weather_road_cond': x['weather_road_cond'].mode(),
  })
)

In [158]:
# head of test_temp
test_temp.head()

,,vehicle_length_cm,vehicle_speed,lane_no,pvehicle_count,elapsed_time,air_temp,realtive_humidity,wind_speed_ms,daylight_cond,weather_road_cond
ID,,,,,,,,,,,
DR_79986,0,574.0,88.000000,1.0,13,546.0,6.615385,95.0,6.307692,daylight,Dry
DR_82800,0,530.0,82.625000,2.0,8,591.0,4.000000,95.0,4.000000,daylight,Wet
DR_93526,0,1687.0,84.000000,2.0,14,576.0,3.000000,95.0,4.000000,daylight,Wet
DR_94423,0,1353.0,84.333333,2.0,12,572.0,8.500000,94.0,5.083333,daylight,Wet
DR_93909,0,597.0,87.357143,2.0,14,481.0,2.000000,93.0,6.000000,daylight,Wet


In [159]:
#Resetting the index of the data frame
test_temp.reset_index()

,ID,level_1,vehicle_length_cm,vehicle_speed,lane_no,pvehicle_count,elapsed_time,air_temp,realtive_humidity,wind_speed_ms,daylight_cond,weather_road_cond
0,DR_79986,0,574.0,88.000000,1.0,13,546.0,6.615385,95.000000,6.307692,daylight,Dry
1,DR_82800,0,530.0,82.625000,2.0,8,591.0,4.000000,95.000000,4.000000,daylight,Wet
2,DR_93526,0,1687.0,84.000000,2.0,14,576.0,3.000000,95.000000,4.000000,daylight,Wet
3,DR_94423,0,1353.0,84.333333,2.0,12,572.0,8.500000,94.000000,5.083333,daylight,Wet
4,DR_93909,0,597.0,87.357143,2.0,14,481.0,2.000000,93.000000,6.000000,daylight,Wet
5,DR_95217,0,586.0,83.866667,2.0,15,554.0,2.400000,92.000000,7.200000,daylight,Wet
6,DR_86089,0,1056.0,81.894737,1.0,19,581.0,5.000000,91.421053,4.947368,twilight,Wet
7,DR_95606,0,1726.0,83.454545,2.0,11,524.0,5.000000,91.000000,0.000000,twilight,Wet
8,DR_95459,0,1526.0,84.176471,1.0,17,514.0,4.058824,91.470588,3.294118,twilight,Wet
9,DR_78768,0,495.0,86.000000,2.0,26,638.0,1.000000,91.346154,5.884615,night,Wet


In [160]:
test_temp.columns

Index(['vehicle_length_cm', 'vehicle_speed', 'lane_no', 'pvehicle_count',
       'elapsed_time', 'air_temp', 'realtive_humidity', 'wind_speed_ms',
       'daylight_cond', 'weather_road_cond'],
      dtype='object')

In [161]:
#dimensions of test_temp
test_temp.shape

(5523, 10)

In [162]:
#Checking missing values in test_temp
test_temp.isnull().sum()

vehicle_length_cm      0
vehicle_speed          0
lane_no                2
pvehicle_count         0
elapsed_time           0
air_temp               0
realtive_humidity      0
wind_speed_ms          0
daylight_cond        643
weather_road_cond    637
dtype: int64

In [163]:
#Removing missing values
test_temp.dropna(axis=0,inplace=True)

In [164]:
#Write test_temp to a csv file
test_temp.to_csv("test_averaged_file_1.csv")

In [165]:
test_temp.shape

(4880, 10)

In [166]:
data.head()

,vehicle_length_cm,vehicle_speed,lane_no,pvehicle_count,elapsed_time,air_temp,realtive_humidity,wind_speed_ms,daylight_cond,weather_road_cond,DrivingStyle
0,1265,87,2.0,19,596,7,95,0,0,0,2
1,1680,82,1.0,11,600,4,95,6,0,0,2
2,936,85,2.0,11,540,1,95,0,0,0,2
3,1503,82,1.0,11,629,3,95,1,0,0,2
4,929,82,1.0,12,558,3,95,3,0,0,2


In [167]:
test_temp.head()

,,vehicle_length_cm,vehicle_speed,lane_no,pvehicle_count,elapsed_time,air_temp,realtive_humidity,wind_speed_ms,daylight_cond,weather_road_cond
ID,,,,,,,,,,,
DR_79986,0,574.0,88.000000,1.0,13,546.0,6.615385,95.0,6.307692,daylight,Dry
DR_82800,0,530.0,82.625000,2.0,8,591.0,4.000000,95.0,4.000000,daylight,Wet
DR_93526,0,1687.0,84.000000,2.0,14,576.0,3.000000,95.0,4.000000,daylight,Wet
DR_94423,0,1353.0,84.333333,2.0,12,572.0,8.500000,94.0,5.083333,daylight,Wet
DR_93909,0,597.0,87.357143,2.0,14,481.0,2.000000,93.0,6.000000,daylight,Wet


In [168]:
test_temp['vehicle_length_cm'] = test_temp['vehicle_length_cm']/100
# test_temp['vehicle_weight_kg'] = test_temp['vehicle_weight_kg']/1000

In [169]:
#Returning unique values of these columns
print(data['weather_road_cond'].unique())
print(data['daylight_cond'].unique())
print(test_temp['weather_road_cond'].unique())
print(test_temp['daylight_cond'].unique())

[0 3 1 2]
[0 2 1]
['Dry' 'Wet' 'Visible tracks' 'Snow covered']
['daylight' 'twilight' 'night']


In [170]:
#Label encoding
le=LabelEncoder()
# label_columns = ['weather_road_cond','prep_type','prep_intensity','daylight_cond']
label_columns = ['weather_road_cond','daylight_cond']
for i in label_columns:
    le.fit(data_holdout[i])
    test_temp[i] = le.transform(test_temp[i])
    test_temp[i] = test_temp[i].astype('float64')

In [ ]:
#Returning unique values of these columns
print(test_temp['weather_road_cond'].unique())
# print(test_temp['prep_type'].unique())
# print(test_temp['prep_intensity'].unique())
print(test_temp['daylight_cond'].unique())

In [ ]:
test_temp.dtypes

In [ ]:
# numerical_columns = [ 'vehicle_length_cm', 'vehicle_weight_kg', 'vehicle_speed', 'air_temp', 'realtive_humidity', 'wind_direction', 'wind_speed_ms', 'elapsed_time']

In [ ]:
# test_temp[numerical_columns] = test_temp[numerical_columns].astype('int64')
# test_temp[numerical_columns] = test_temp[numerical_columns].astype('float64')

In [ ]:
# test_temp[categorical_columns] = test_temp[categorical_columns].astype('float64')

In [ ]:
# test_temp.dtypes

In [ ]:
test_temp.columns

In [ ]:
# test_temp.columns = ['0','1','2','3','4','5','6','7','8','9','10','11','12','13',]

In [ ]:
test_temp.head()

In [ ]:
train_split_X.head()

## Model Building

#### 1. Decision trees

In [ ]:
def decision_tree():
    print("-"*30,"Decision_Tree")
    # Creating an object for decision Trees
#     clf = DecisionTreeClassifier(criterion='gini',class_weight='balanced')
    clf = DecisionTreeClassifier(max_depth=6,max_features='auto',min_samples_leaf=10,min_samples_split=3,criterion='gini')
    #Using the object created for decision Trees to fit on train data
    clf.fit(train_split_X, train_split_Y)
    
    feat_importance(clf.feature_importances_,"Decision Tree")

    #Using the object created for decision Trees to predict on validation data
    y_pred_dt = clf.predict(val_split_X)

    #Using the object created for decision Trees to predict on test data
#     test_y_pred_dt = clf.predict(test_data_cluster_4)

    #Calculating the accuracy of train data
    acc_dt_train = round( clf.score(train_split_X, train_split_Y) * 100, 2)
    print ('train_accuracy_dt: ' +' '+str(acc_dt_train) + ' percent')

    #Calculating the accuracy of validation data
    acc_dt_validation = accuracy_score(val_split_Y, y_pred_dt)
    print('validation_accuracy_dt: '+str(round((acc_dt_validation)*100,2)) + 'percent')
    print(classification_report(val_split_Y, y_pred_dt))
#     dt_precision,dt_recall,dt_fscore,dt_support = precision_recall_fscore_support(val_split_Y, y_pred_dt, average='micro',labels=[1])
#     print("Recall: ",dt_recall)
#     print("Precision: ",dt_precision)
# nestimator,criterion,spliiter,classweight,min_sample_split
    test_pred = clf.predict(test_temp)
    return test_pred

In [ ]:
def feat_importance(best_features,algo):
    col = train_split_X.columns
    #modelname.feature_importance_
    y = best_features
    #plot
    fig, ax = plt.subplots() 
    width = 0.4 # the width of the bars 
    ind = np.arange(len(y)) # the x locations for the groups
    ax.barh(ind, y, width, color="green")
    ax.set_yticks(ind+width/10)
    ax.set_yticklabels(col, minor=False)
    plt.title('Feature importance in '+algo)
    plt.xlabel('Relative importance')
    plt.ylabel('feature') 
    plt.figure(figsize=(5,7))
    fig.set_size_inches(6.5, 9.5, forward=True)

#### 2. random  forest

In [ ]:
def random_forest():
    print("-"*30,"Random Forest")
    # Creating an object for Random Forest
    clf = RandomForestClassifier(n_estimators= 1000, min_samples_split= 2, min_samples_leaf= 1, max_features= 'auto', max_depth= 50, bootstrap= False,n_jobs=-1)

    #Using the object created for Random Forest to fit on train data
    clf.fit(train_split_X, train_split_Y)

    #checking the feature importance
    feat_importance(clf.feature_importances_,"Random Forest")
    
    #Using the object created for Random Forest to predict on validation data
    y_pred_rf = clf.predict(val_split_X)

    #Using the object created for Random Forest to predict on test data
#     test_y_pred_rf = clf.predict(test_data_cluster_4)

    #Calculating the accuracy of train data
    acc_rf_train = round( clf.score(train_split_X, train_split_Y) * 100, 2)
    print ('train_accuracy_rf: ' +' '+str(acc_rf_train) + ' percent')

    #Calculating the accuracy of validation data
    acc_rf_validation = accuracy_score(val_split_Y, y_pred_rf)
    print('validation_accuracy_rf: '+str(round((acc_rf_validation)*100,2)) + 'percent')
    print(classification_report(val_split_Y, y_pred_rf))
#     rf_precision,rf_recall,rf_fscore,rf_support = precision_recall_fscore_support(val_split_Y, y_pred_rf, average='micro',labels=[1])
#     print("Recall: ",rf_recall)
#     print("Precision: ",rf_precision)
    test_pred = clf.predict(test_temp)
    return test_pred

#### 3. Adaboost

In [ ]:
def adaboost():
    print("-"*30,"Adaboost")
    # Creating an object for Adaboost
    clf = AdaBoostClassifier()

    #Using the object created for Adaboost to fit on train data
    clf.fit(train_split_X, train_split_Y)

    #Using the object created for Adaboost to predict on validation data
    y_pred_adaboost = clf.predict(val_split_X)
    feat_importance(clf.feature_importances_,"ADABoost")

    #Using the object created for Adaboost to predict on test data
#     test_y_pred_adaboost = clf.predict(test_data_cluster_4)

    #Calculating the accuracy of train data
    acc_adaboost_train = round( clf.score(train_split_X, train_split_Y) * 100, 2)
    print ('train_accuracy_adaboost: ' +' '+str(acc_adaboost_train) + ' percent')

    #Calculating the accuracy of validation data
    acc_adaboost_validation = accuracy_score(val_split_Y, y_pred_adaboost)
    print('validation_accuracy_adaboost: '+str(round((acc_adaboost_validation)*100,2)) + 'percent')
    print(classification_report(val_split_Y, y_pred_adaboost))
#     print("Recall: ",ada_recall)
#     print("Precision: ",ada_precision)
    test_pred = clf.predict(test_temp)
    return test_pred

#### 4. Stochastic gradient descent

In [ ]:
def sgd():
    print("-"*30,"SGD")
    # Creating an object for Stochastic gradient decsent
    clf = SGDClassifier(max_iter=5,n_jobs=-1,)

    #Using the object created for Stochastic gradient decsent to fit on train data
    clf.fit(train_split_X, train_split_Y)

    #Using the object created for Stochastic gradient decsent to predict on validation data
    y_pred_sgd = clf.predict(val_split_X)
#     feat_importance(clf.feature_importances_,"SGBoost")

    #Using the object created for Stochastic gradient decsent to predict on test data
#     test_y_pred_sgd = clf.predict(test_data_cluster_4)

    #Calculating the accuracy of train data
    acc_sgd_train = round( clf.score(train_split_X, train_split_Y) * 100, 2)
    print ('train_accuracy' +' '+str(acc_sgd_train) + ' percent')

    #Calculating the accuracy of validation data
    acc_sgd_validation = accuracy_score(val_split_Y, y_pred_sgd)
    print('validation_accuracy '+str(round((acc_sgd_validation)*100,2)) + 'percent')
    print(classification_report(val_split_Y, y_pred_sgd))

#     print(precision_recall_fscore_support(val_split_Y, acc_sgd_validation))
#     print("Recall: ",sgd_recall)
#     print("Precision: ",sgd_precision)
    test_pred = clf.predict(test_temp)
    return test_pred

#### 5. XGBoost

In [ ]:
def xgboost():
    print("-"*30,"XGBoost")
    # Creating an object for XGBoost
    clf = XGBClassifier(max_iter=5, tol=None,n_jobs=-1,warm)

    #Using the object created for XGBoost to fit on train data
    clf.fit(train_split_X, train_split_Y,)

    #Using the object created for XGBoost to predict on validation data
    y_pred_xgb = clf.predict(val_split_X)
    feat_importance(clf.feature_importances_,"XGBoost")
#     plt.bar(range(len(clf.feature_importances_)), clf.feature_importances_)
#     pyplot.show()
#     plot_importance(clf)
#     plt.show()

    #Using the object created for XGBoost to predict on test data
#     test_y_pred_sgd = clf.predict(test_data_cluster_4)

    #Calculating the accuracy of train data
    acc_xgb_train = round( clf.score(train_split_X, train_split_Y) * 100, 2)
    print ('train_accuracy' +' '+str(acc_xgb_train) + ' percent')

    #Calculating the accuracy of validation data
    acc_xgb_validation = accuracy_score(val_split_Y, y_pred_xgb)
    print('validation_accuracy '+str(round((acc_xgb_validation)*100,2)) + 'percent')
    print(classification_report(val_split_Y, y_pred_xgb))
#     print(precision_recall_fscore_support(val_split_Y, acc_sgd_validation))
#     print("Recall: ",sgd_recall)
#     print("Precision: ",sgd_precision)
#     print(confusion_matrix(y_pred_sgd, val_split_Y))
#     predictions = [value for value in val_split_Y]
#     accuracy = accuracy_score(y_pred_sgd, predictions)
#     precision = precision_score(y_pred_sgd, predictions,average='micro')
#     recall = recall_score(y_pred_sgd, predictions,average='micro')
#     f1 = f1_score(y_pred_sgd, predictions,average='micro')
#     print("Accuracy_score: %.2f%% on test dataset" % (accuracy * 100.0))
#     print("precision_score: %.2f%% on test dataset" % (precision * 100.0))
#     print("recall_score: %.2f%% on test dataset" % (recall * 100.0))
#     print("f1_score: %.2f%% on test dataset" % (f1 * 100.0))
# # print("roc_auc test set", roc_auc_score(y_test, model.predict_proba(X_test)[:,1]))
# # print("roc_auc training set", roc_auc_score(y_train, model.predict_proba(X_train)[:,1]))
    test_pred = clf.predict(test_temp)
    return test_pred

#### 6. Catboost

In [ ]:
def catboost():
#     params = {'depth':[3,1,2,6,4,5,7,8,9,10],
#           'iterations':[250,100,500,1000],
#           'learning_rate':[0.03,0.001,0.01,0.1,0.2,0.3], 
#           'l2_leaf_reg':[3,1,5,10,100],
#           'border_count':[32,5,10,20,50,100,200],
#           'ctr_border_count':[50,5,10,20,100,200],
#           'thread_count':4
#              }
    clf = CatBoostClassifier(iterations=2, learning_rate=0.5, depth=2, loss_function='MultiClass')
    # Fit model
    #Using the object created for catboost to fit on train data
#     gsearch2b = GridSearchCV(estimator = clf, scoring='recall',n_jobs=-1,iid=False, cv=5,verbose=3,param_grid=params)
    clf.fit(train_split_X, train_split_Y)
#     clf.fit(val_split_X, val_split_Y)
#     print("gsearch2b.best_params_",gsearch2b.best_params_)

#     print(classification_report(val_split_Y, y_pred_rf))

    #Using the object created for catboost to predict on validation data
    y_pred_cat = clf.predict(val_split_X)
    
    #Using the object created for catboost to predict on test data
#     test_y_pred_sgd = clf.predict(test_data_cluster_4)

    #Calculating the accuracy of train data
    acc_cat_train = round( clf.score(train_split_X, train_split_Y) * 100, 2)
    print ('train_accuracy' +' '+str(acc_cat_train) + ' percent')

    #Calculating the accuracy of validation data
    acc_cat_validation = accuracy_score(val_split_Y, y_pred_cat)
    print('validation_accuracy '+str(round((acc_cat_validation)*100,2)) + 'percent')
    print(classification_report(val_split_Y, y_pred_cat))
#     print(precision_recall_fscore_support(val_split_Y, acc_sgd_validation))
#     print("Recall: ",sgd_recall)
#     print("Precision: ",sgd_precision)
#     print(confusion_matrix(y_pred_cat, val_split_Y))
#     predictions = [value for value in val_split_Y]
    accuracy = accuracy_score(val_split_Y,y_pred_cat)
#     precision = precision_score(y_pred_cat, predictions,average='micro')
#     recall = recall_score(y_pred_cat, predictions,average='micro')
#     f1 = f1_score(y_pred_cat, predictions,average='micro')
    print("Accuracy_score: %.2f%% on test dataset" % (accuracy * 100.0))
#     print("precision_score: %.2f%% on test dataset" % (precision * 100.0))
#     print("recall_score: %.2f%% on test dataset" % (recall * 100.0))
#     print("f1_score: %.2f%% on test dataset" % (f1 * 100.0))
# print("roc_auc test set", roc_auc_score(y_test, model.predict_proba(X_test)[:,1]))
# print("roc_auc training set", roc_auc_score(y_train, model.predict_proba(X_train)[:,1]))
    test_pred = clf.predict(test_temp)
    return test_pred

#### 7. LightGBM

In [ ]:
def lightgbm():
#     lgb_train = lgb.Dataset(train_split_X, train_split_Y)
#     lgb_eval = lgb.Dataset(val_split_X, val_split_Y, reference=lgb_train)
#     params = {
#     'boosting_type': 'gbdt',
#     'objective': 'multiclass',
#     'metric': 'multi_logloss',
#     'num_leaves': 31,
#     'learning_rate': 0.05,
#     'feature_fraction': 0.9,
#     'bagging_fraction': 0.8,
#     'bagging_freq': 5,
#     'verbose': 0
#     }
#     clf = lgb.train(train_split_X, 2000, valid_sets=[train_split_X, train_split_Y], early_stopping_rounds=100, verbose_eval=200)
    # Creating an object for LightGBM
    gbm = lgb.LGBMClassifier(num_leaves=31,learning_rate=0.05,n_estimators=10,feature_fraction= 0.9,bagging_fraction= 0.8,bagging_freq=5,verbose=1)
   # Using the object created for LightGBM to fit on train data
    gbm.fit(train_split_X, train_split_Y,eval_set=[(val_split_X, val_split_Y)],eval_metric='multi_logloss')

    print('Starting predicting...')
    # Using the object created for LightGBM to predict on validation data
    y_pred_gbm = gbm.predict(val_split_X, num_iteration=gbm.best_iteration_)
#     feat_importance(gbm.best_iteration_,"LightGBM")
    # eval
#     predictions = [value for value in val_split_Y]
    accuracy = accuracy_score(val_split_Y, y_pred_gbm)
#     precision = precision_score(val_split_Y, y_pred,average='micro')
#     recall = recall_score(val_split_Y, y_pred,average='micro')
    print("Accuracy_score: %.2f%% on test dataset" % (accuracy * 100.0))
#     print("precision_score: %.2f%% on test dataset" % (precision * 100.0))
#     print("precision_score: %.2f%% on test dataset" % (recall * 100.0))
#     # feature importances
#     print('Feature importances:', list(gbm.feature_importances_))
    print(classification_report(val_split_Y, y_pred_gbm))
    test_pred = gbm.predict(test_temp, num_iteration=gbm.best_iteration_)
    return test_pred

#### 8. Stacking

In [ ]:
def stack():
    print("-"*30,"Voting Classifier")
    model1 = lgb.LGBMClassifier(num_leaves=31,learning_rate=0.005,n_estimators=20,feature_fraction= 0.9,bagging_fraction= 0.8,bagging_freq=5,verbose=1)
    model2 = DecisionTreeClassifier(max_depth=3,max_features='auto',min_samples_leaf=10,min_samples_split=3,criterion='gini')
#     model3 = XGBClassifier(max_iter=5, tol=None,n_jobs=-1,objective='multi:softprob')
    model4 = CatBoostClassifier(iterations=2, learning_rate=0.5, depth=6, loss_function='MultiClass')
    model3 = XGBClassifier(n_estimators=15, learning_rate=0.05, max_depth=4, min_samples_leaf=6)
    model = VotingClassifier(estimators=[('LGBM', model1), ('DT', model2), ('XGB', model3), ('CAT',model4)], voting='soft', flatten_transform=True,n_jobs=-1)
    model.fit(train_split_X,train_split_Y)
    stack_pred = model.predict(val_split_X)
    test_pred = model.predict(test_temp)
    print(classification_report(val_split_Y, stack_pred))
#     model.score(val_split_Y, stack_pred)*100
    accuracy = accuracy_score(val_split_Y, stack_pred)
    print("Accuracy_score: %.2f%% on test dataset" % (accuracy * 100.0))
    print(confusion_matrix(val_split_Y, stack_pred))
    return test_pred

In [ ]:
dt_test_pred = decision_tree()
rf_test_pred = random_forest()
ada_test_pred = adaboost()
sgd_test_pred = sgd()
xg_test_pred = xgboost()
cat_test_pred = catboost()
lgb_test_pred = lightgbm()
stack_test_pred =  stack()
# pred = pd.DataFrame(pred)
# print("Stack",pred[0].value_counts())

In [ ]:
import graphviz
from sklearn import tree
import pydotplus
import os

os.environ["PATH"] += os.pathsep + 'D:\\Program Files (x86)\\Graphviz2.38\\bin\\'

In [ ]:
from sklearn.externals.six import StringIO  
from IPython.display import Image  
from sklearn.tree import export_graphviz
import pydotplus
import pydot

In [ ]:
clf = DecisionTreeClassifier(max_depth=6,max_features='auto',min_samples_leaf=10,min_samples_split=3,criterion='gini')
clf.fit(train_split_X, train_split_Y)

In [ ]:
dot_data = StringIO()
tree.export_graphviz(clf, out_file=dot_data)
dot_data = tree.export_graphviz(clf, out_file=None) 
graph = graphviz.Source(dot_data) 

In [ ]:
dt_graph = export_graphviz(clf, out_file=None, 
                feature_names = train_split_X.columns,
                class_names = ['Aggresive','Normal','Vague'],
                rounded = True, proportion = False, 
                precision = 2, filled = True)

# Convert to png using system command (requires Graphviz)
# from subprocess import call
# call(['dot', '-Tpng', 'tree.dot', '-o', 'tree.png', '-Gdpi=600'])

# # Display in jupyter notebook
# from IPython.display import Image
# Image(filename = 'tree.png')
graph = graphviz.Source(dt_graph)
graph

In [ ]:
stack_test_pred = pd.DataFrame(stack_test_pred)
stack_test_pred[0].value_counts()

In [ ]:
dt_test_pred = pd.DataFrame(dt_test_pred)
dt_test_pred[0].value_counts()

In [ ]:
rf_test_pred = pd.DataFrame(rf_test_pred)
rf_test_pred[0].value_counts()

In [ ]:
ada_test_pred = pd.DataFrame(ada_test_pred)
ada_test_pred[0].value_counts()

In [ ]:
sgd_test_pred = pd.DataFrame(sgd_test_pred)
sgd_test_pred[0].value_counts()

In [ ]:
xg_test_pred = pd.DataFrame(xg_test_pred)
xg_test_pred[0].value_counts()

In [ ]:
cat_test_pred = pd.DataFrame(cat_test_pred)
cat_test_pred[0].value_counts()

In [ ]:
lgb_test_pred = pd.DataFrame(lgb_test_pred)
lgb_test_pred[0].value_counts()

In [ ]:
stack_test_pred_holdup = stack_test_pred
stack_test_pred_holdup[0].value_counts()

In [ ]:
test_pred = stack_test_pred

In [ ]:
test_pred[0].value_counts()

In [ ]:
pred_temp = test_pred[0]

In [ ]:
submission = pd.DataFrame(test_data['ID'])

In [ ]:
submission['DrivingStyle'] = pred_temp

In [ ]:
submission.head()

In [ ]:
submission['DrivingStyle'].value_counts()

In [ ]:
submission.to_csv("tuned_submission1.csv",index=None)

In [ ]:
val_split_Y.value_counts()

In [ ]:
# temp_data['weather_road_cond'] = temp_data['weather_road_cond'].fillna('missing')

In [ ]:
# temp_data['prep_type'] = temp_data['prep_type'].fillna('missing')

In [ ]:
# temp_data['prep_intensity'].dtype

In [ ]:
# temp_data['prep_intensity'] = temp_data['prep_intensity'].astype('category')

In [ ]:
# temp_data['prep_intensity'].cat.add_categories('missing')

In [ ]:
# temp_data['prep_intensity'] = temp_data['prep_intensity'].fillna('missing')

In [ ]:
index_list = temp_data[temp_data['weather_road_cond'].isna()].index

In [ ]:
for i in tqdm_notebook(index_list):
    temp_data['weather_road_cond'].iloc[i] = temp_data['weather_road_cond'].iloc[i-1]
    temp_data['prep_type'].iloc[i] = temp_data['prep_type'].iloc[i-1]
    temp_data['prep_intensity'].iloc[i] = temp_data['prep_intensity'].iloc[i-1]
    temp_data['daylight_cond'].iloc[i] = temp_data['daylight_cond'].iloc[i-1]

In [ ]:
temp_data.isnull().sum()

In [ ]:
temp_data['weather_road_cond'] = temp_data['weather_road_cond'].fillna(temp_data['weather_road_cond'].mode()[0])
temp_data['prep_type'] = temp_data['prep_type'].fillna(temp_data['prep_type'].mode()[0])
temp_data['prep_intensity'] = temp_data['prep_intensity'].fillna(temp_data['prep_intensity'].mode()[0])
temp_data['daylight_cond'] = temp_data['daylight_cond'].fillna(temp_data['daylight_cond'].mode()[0])
temp_data['lane_no'] = temp_data['lane_no'].fillna(temp_data['lane_no'].mode()[0])

In [ ]:
temp_data.isnull().sum()

In [ ]:
temp_data.to_csv("averaged_data_file.csv",sep=",",index=None)

In [ ]:
temp_data['DrivingStyle'].value_counts()